# Modelo Preditivo de Fraude
Random Forest sem Cross validation com MLLIB e modleo salvo

## Objetivo
Os POCs reportados são focados em agências com maior ocorrência de fraude, não constituindo, portanto, um registro completo dos POCs ocorridos. Esta análise tem por objetivo  a melhorar a performance do modelo preditivo, a partir da marcação dos POCs de forma mais completa, por meio da análise dos eventos ocorridos nas agências/ATMs com POCs reportados.

## Libraries

In [2]:
import pyspark
from pyspark import SparkConf
from pyspark import SparkContext
from pyspark.sql import SQLContext
from pyspark.sql import HiveContext
hive_context = HiveContext(sc)
from pyspark.sql.types import Row
from pyspark.sql.types import *
from pyspark.sql.types import DoubleType
from pyspark.sql.types import DateType
from pyspark.sql.types import IntegerType
from pyspark.sql.types import TimestampType
from pyspark.sql.functions import *
#import matplotlib as mpl
#import matplotlib.pyplot as plt
import atexit
from numpy import array
import numpy as np
import pandas as pd
from pyspark.mllib.linalg import Vectors
from pylab import plot,show,hist,figure,title
import unicodedata
from datetime import datetime, time
from pyspark.sql.functions import col,udf, unix_timestamp
from pyspark.ml import Pipeline
from pyspark.ml.feature import HashingTF, Tokenizer
from pyspark.ml.classification import DecisionTreeClassifier
from pyspark.ml.feature import StringIndexer, VectorIndexer
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml import Pipeline

# para plots
%matplotlib inline

NameError: name 'sc' is not defined

## constantes

In [287]:
mediana = 50  # mediana da ocorrência de eventos diários (feature)
percentil_max = 95 # percentil máximo considerado para treinar o modelo
D_90 = 90 # 90 dias a subtrair (usado somente para treino inicial)
D_1 = 1 # obtem eventos de D-1 para previsão de POC em produção
D_2 = 2 # obtem eventos a aprtir de D-2 para trás para treino constante do modelo em produção

## Carga de Dados Brutos Hadoop Produção

In [288]:
####################################################################################
# PARA RODAR EM HADOOP PRODUCAO DESCOMENTAR ABAIXO, COMENTANDO A CELULA DO MAC OSX 
####################################################################################
funcStrToDate_mdyy =  udf (lambda x: datetime.strptime(x, '%Y-%m-%d %H:%M:%S.%f')\
                           .replace(minute=0, hour=0, second=0, microsecond=0), DateType())
funcStrToDate_dmyyyy =  udf (lambda x: datetime.strptime(x, '%m/%d/%y'), DateType())

In [289]:
rdd_eventlog = hive_context.table('default.ext_eventos_fullog1908')
poc_in = '/user/x181472/export/BASE_POC_ATM_090816.csv'
#poc_in = '/temporario/V6_BASE_POC_ATM_060916.csv'

tranlog_in = hive_context.table('default.ext_juntatudo12')
tabelao_out = '/temporario/tabelao-temp-V5.2.5.csv'
tabelao_in = '/user/x184166/tabelao-temp-V5.2.5.csv' # Hermann
poceven_in = '/temporario/pokeventos-V2.csv'

In [290]:
df_eventlog = rdd_eventlog \
    .map(lambda p: (p[1],p[2],p[3], p[4])) \
    .toDF(['nr_sequ_even',
           'dh_even', 
           'cd_eqpt', 
           'nr_idef_secu' 
                   ])
df_eventlog.printSchema() 

print ("\n\nNumero de linhas carregadas: " + str("%d" % (rdd_eventlog.count())))

# sumarizar a base por agencia+atm+evento+dia
df_eventlog = df_eventlog.withColumn('dh_even', funcStrToDate_mdyy(col('dh_even'))) 

df_eventlog = df_eventlog\
    .groupBy(['nr_idef_secu', 'cd_eqpt', 'nr_sequ_even', 'dh_even']).count()\
    .orderBy(col('nr_idef_secu'), col('cd_eqpt'), col('nr_sequ_even'), col('dh_even').desc())\
    .withColumnRenamed('count', 'nr_eventos')

root
 |-- nr_sequ_even: long (nullable = true)
 |-- dh_even: string (nullable = true)
 |-- cd_eqpt: string (nullable = true)
 |-- nr_idef_secu: string (nullable = true)



Numero de linhas carregadas: 18341921


In [295]:
#df_eventlog.limit(10).toPandas()

## Carga de dados Brutos MAC

In [296]:
####################################################################################
# PARA RODAR MAC OSX DESCOMENTAR ABAIXO, COMENTANDO A CELULA DO HADOOP PRODUCAO 
####################################################################################
#eventlog_in = '/Users/josecassiodossantosjunior/Documents/spark-2.0.0-bin-hadoop2.7/MySpark/dados/tbfullog.csv'
#poc_in = '/Users/josecassiodossantosjunior/Documents/spark-2.0.0-bin-hadoop2.7/MySpark/dados/BASE_POC_ATM_090816.csv'
#ranlog_in = '/Users/josecassiodossantosjunior/Documents/spark-2.0.0-bin-hadoop2.7/dados/juntatudo11pordia-tranfin.csv'
#tabelao_out = '/Users/josecassiodossantosjunior/Documents/spark-2.0.0-bin-hadoop2.7/MySpark/dados/tabelao_stdr-V5.2.2.csv'
#tabelao_in = '/Users/josecassiodossantosjunior/Documents/spark-2.0.0-bin-hadoop2.7/MySpark/dados/tabelao_stdr-V5.2.2.csv'
#previsao = '/Users/josecassiodossantosjunior/Documents/spark-2.0.0-bin-hadoop2.7/MySpark/dados/previsao-V5.2.2.csv'
#poceven_in = '/Users/josecassiodossantosjunior/Documents/spark-2.0.0-bin-hadoop2.7/MySpark/dados/pokeventos.csv'

#funcStrToDate_mdyy =  udf (lambda x: datetime.strptime(x, '%d/%m/%Y'), DateType())
#funcStrToDate_dmyyyy =  udf (lambda x: datetime.strptime(x, '%m/%d/%y'), DateType())

#rdd_eventlog = sc.textFile(eventlog_in)
#print ("\n\nNúmero de linhas carregadas: " + str("%d" % (rdd_eventlog.count())))

#header = rdd_eventlog.first()

#df_eventlog = rdd_eventlog \
#    .filter(lambda line: line != header) \
#    .map(lambda k: k.split(",")) \
#   .map(lambda p: (p[1],p[2],p[3],p[4], p[5])) \
#    .toDF(['nr_sequ_even',
#            'dh_even', 
#            'cd_eqpt', 
#            'nr_idef_secu', 
#            'nr_eventos'
#                  ])
#df_eventlog.printSchema()    

In [297]:
#df_eventlog = df_eventlog.withColumn('dh_even', funcStrToDate_mdyy(col('dh_even'))) 

In [298]:
df_eventlog = df_eventlog \
    .withColumn('nr_sequ_even', df_eventlog["nr_sequ_even"].cast(StringType()))\
    .withColumn('cd_eqpt', df_eventlog["cd_eqpt"].cast(StringType()))\
    .withColumn('nr_idef_secu', df_eventlog["nr_idef_secu"].cast(StringType())) \
    .withColumn('dh_even', df_eventlog["dh_even"].cast(TimestampType()))\
    .withColumn('nr_eventos', df_eventlog["nr_eventos"].cast(IntegerType()))

df_eventlog.printSchema()   

root
 |-- nr_idef_secu: string (nullable = true)
 |-- cd_eqpt: string (nullable = true)
 |-- nr_sequ_even: string (nullable = true)
 |-- dh_even: timestamp (nullable = true)
 |-- nr_eventos: integer (nullable = false)



In [299]:
#df_eventlog.limit(10).orderBy(col("nr_eventos").desc()).toPandas()

In [300]:
# DEPOIS: acertar abaixo para pegar direto ultima data do arquivo original como data_ref
# Filtrando últimos 90 dias
import datetime, time
from dateutil.relativedelta import relativedelta

# obtendo faixa do período de histórico a partir da data de referencia
data_ref = datetime.datetime(2016, 8, 10) 
D_menos1 = data_ref + relativedelta(days=-D_1)
D_menos2 = data_ref + relativedelta(days=-D_2)
D_menos90 = D_menos2 + relativedelta(days=-D_90)
print "D-90:   " + str(D_menos90)
print "D-1 :   " + str(D_menos1)
print "D-2 :   " + str(D_menos2)

D-90:   2016-05-10 00:00:00
D-1 :   2016-08-09 00:00:00
D-2 :   2016-08-08 00:00:00


In [301]:
dt_sql_90 = (time.mktime(datetime.datetime.strptime(str(D_menos90.date()), "%Y-%m-%d").timetuple()))
dt_sql_1 = (time.mktime(datetime.datetime.strptime(str(D_menos1.date()), "%Y-%m-%d").timetuple()))
dt_sql_2 = (time.mktime(datetime.datetime.strptime(str(D_menos2.date()), "%Y-%m-%d").timetuple()))
dt_sql_ref = (time.mktime(datetime.datetime.strptime(str(data_ref.date()), "%Y-%m-%d").timetuple()))
print dt_sql_90
print dt_sql_1
print dt_sql_2
print dt_sql_ref

1462849200.0
1470711600.0
1470625200.0
1470798000.0


In [302]:
# FILTRANDO SOMENTE ULTIMOS 90 DIAS, a partir de D-2
df_treinoevent = df_eventlog.where("CAST(dh_even AS INT) BETWEEN {0} AND {1}"\
#                                   .format(dt_sql_90, dt_sql_2)).orderBy("dh_even")
#df_treinoevent.limit(10).toPandas()

SyntaxError: invalid syntax (<ipython-input-302-96063a80fe96>, line 3)

In [ ]:
print ("\nNúmero de linhas carregadas: " + str("%d" % (df_treinoevent.count())))
# ABAIXO: COMPOPRTAMENTO ESTRANHO - traz mais linhas que o numero original apos dropna??!!
#print ("\nNúmero de linhas após retirar NAs: " + str("%d" % (df_treinoevent.dropna().count())))
#df_treinoevent = df_treinoevent.dropna()

In [198]:
df_treinoevent.filter((col("nr_sequ_even") == '3321')).count()

0

In [15]:
#print ("\nNúmero de linhas carregadas: " + str("%d" % (df_treinoevent.count())))

## Obtendo Eventos Marcadores de POC (selecionados como features)
Carrega a lista de eventos marcadores de POC a partir de arquivo já gerado pelo notebook STDR-Seletor de Pokeventos.

In [88]:
# lendo arquivo de pokeventos e carregando em lista
pd_poceven = pd.read_csv(poceven_in, header=None).T
pd_poceven.columns = ['pokevento']

l_even = pd_poceven.pokevento.tolist()

def foo(l_even, dtype=str):
    return map(dtype, l_even)
l_evenpoc = foo(l_even)
l_features = l_evenpoc

In [89]:
l_features

['4266',
 '899',
 '2034',
 '4268',
 '2982',
 '3260',
 '4269',
 '1641',
 '1642',
 '3425',
 '1648',
 '2422',
 '1489',
 '679',
 '723',
 '724',
 '2761',
 '3814',
 '6776',
 '2267',
 '6778',
 '5160',
 '5278',
 '569',
 '4274',
 '2493',
 '1765',
 '791',
 '1650',
 '1769',
 '1813',
 '799',
 '1656',
 '3321',
 '5104',
 '684',
 '2433',
 '2434',
 '5558',
 '687',
 '6944',
 '2930',
 '5559',
 '6332',
 '5109',
 '6333',
 '689',
 '6334',
 '6336',
 '6787',
 '4556',
 '6338',
 '3666',
 '4557',
 '2163',
 '3829',
 '2165',
 '3056',
 '3551',
 '4109',
 '5171',
 '5172',
 '626',
 '627',
 '628',
 '4609',
 '629',
 '3894',
 '5720',
 '5721',
 '2392',
 '3283',
 '3896',
 '3446',
 '853',
 '3286',
 '3899',
 '5563',
 '3288',
 '6500',
 '900',
 '6790',
 '6341',
 '4560',
 '6342',
 '6793',
 '4110',
 '4561',
 '4562',
 '1601',
 '3672',
 '4563',
 '5292',
 '1892',
 '4610',
 '1606',
 '632',
 '3227',
 '633',
 '1608',
 '4119',
 '1609',
 '4615',
 '2177',
 '6732',
 '4456',
 '6734',
 '5232',
 '3004',
 '4185',
 '3006',
 '3952',
 '5239',
 

In [27]:
# prepara lista de features para eventos se usar SQL ao inves de Pandas
#for i in range(len(l_features)):
#    l_features[i] = "ev" + str(l_features[i])

#l_features

In [90]:
# filtrando log para somente 50 eventos considerados como features
df_treinoevent = df_treinoevent.filter((col("nr_sequ_even").isin(l_features)))

In [91]:
df_treinoevent.limit(10).toPandas()

nr_idef_secu cd_eqpt nr_sequ_even    dh_even  nr_eventos
0         1227     170         4269 2016-05-24           1
1         1320   10001         3260 2016-05-24           1
2         1320     151         2982 2016-05-24           1
3         1320     151         4266 2016-05-24           1
4         1320     151         4269 2016-05-24           1
5         1320     152         2034 2016-05-24           5
6         1320     152         2982 2016-05-24           1
7         1320     152         4266 2016-05-24           1
8         1320     152         4269 2016-05-24           1
9         1320     153         2034 2016-05-24           3

In [92]:
# calculando percentis de baseline para todas as observações do log com base em agencia/atm/evento
# OBS: o uso do mediana desta forma permite usar os eventos isoladamente como marcadores 
# pois normaliza-os, tornando este marcador independente das ATM onde ocorrem
df_eventlog_percen = df_treinoevent.withColumn('id', concat(col("nr_idef_secu"), lit("-"), 
                                             col("cd_eqpt"), lit("-"),col("nr_sequ_even")))\
                                .select(['id', 'nr_eventos'])

In [93]:
#df_eventlog_percen.take(5)

In [94]:
rdd_eventlog_percen = df_eventlog_percen.rdd \
    .map(lambda x: (x[0], (x[1],), )) \
    .reduceByKey(lambda acc, val: acc + val) \
    .map(lambda x: (x[0], np.percentile(x[1], mediana), np.percentile(x[1], percentil_max)))

In [95]:
rdd_eventlog_percen = rdd_eventlog_percen \
    .map(lambda k: (k[0].split("-"), k[1], k[2])) 

In [96]:
df_eventlog_percen = rdd_eventlog_percen \
    .map(lambda p: (p[0][0],p[0][1],p[0][2],float(p[1]), float(p[2])  )) \
    .toDF(['nr_idef_secu',
           'cd_eqpt',
           'nr_sequ_even',
           'mediana',
           'percentil_max'
                   ])
#df_eventlog_percen.limit(10).toPandas()

In [97]:
df_eventlog_percen.printSchema()

root
 |-- nr_idef_secu: string (nullable = true)
 |-- cd_eqpt: string (nullable = true)
 |-- nr_sequ_even: string (nullable = true)
 |-- mediana: double (nullable = true)
 |-- percentil_max: double (nullable = true)



In [98]:
#df_eventlog.count()

In [99]:
#df_eventlog.limit(10).toPandas()

In [100]:
# colocar no log total as colunas de mediana e percentil máximo por agencia+atm+evento 
df_eventlog2 = df_treinoevent \
    .join(df_eventlog_percen, (df_treinoevent['nr_idef_secu'] == df_eventlog_percen['nr_idef_secu']) &
                              (df_treinoevent['cd_eqpt'] == df_eventlog_percen['cd_eqpt']) &
                              (df_treinoevent['nr_sequ_even'] == df_eventlog_percen['nr_sequ_even']), 'left') \
    .drop(df_eventlog_percen["nr_idef_secu"])\
    .drop(df_eventlog_percen["cd_eqpt"])\
    .drop(df_eventlog_percen["nr_sequ_even"])

In [101]:
df_eventlog2.limit(10).toPandas()

nr_idef_secu cd_eqpt nr_sequ_even    dh_even  nr_eventos  mediana  \
0          105   10001         3260 2016-06-15           1      1.0   
1          105   10001         3260 2016-08-06           1      1.0   
2          105   10001         3260 2016-06-17           1      1.0   
3          105   10001         3260 2016-06-09           1      1.0   
4          105   10001         3260 2016-06-01           1      1.0   
5          105   10001         3260 2016-07-09           1      1.0   
6          105   10001         3260 2016-06-16           1      1.0   
7          105   10001         3260 2016-07-21           1      1.0   
8          105   10001         3260 2016-05-28           1      1.0   
9          105   10001         3260 2016-08-02           1      1.0   

   percentil_max  
0            1.0  
1            1.0  
2            1.0  
3            1.0  
4            1.0  
5            1.0  
6            1.0  
7            1.0  
8            1.0  
9            1.0

In [102]:
#df_eventlog2.count()

In [103]:
# tirando zeros a esquerda e truncando codigos de agencia e atm
df_eventlog2 = df_eventlog2\
    .withColumn('cd_eqpt', trim(df_eventlog2.cd_eqpt))\
    .withColumn('nr_idef_secu', trim(df_eventlog2.nr_idef_secu))\
    .withColumn('nr_idef_secu', regexp_replace('nr_idef_secu', '^0+(?!$)', '')) # removendo zero `esquerda
#    .withColumn('nr_idef_secu', regexp_replace('nr_idef_secu', '^0', '')) # removendo zero `esquerda
# testar s.replaceFirst("^0+(?!$)", "")

In [104]:
#df_eventlog2.orderBy('nr_sequ_even').limit(10).toPandas()

### Dados de registro de POC
Obtém os POCs de Retenção de Cartão, somente para os últimos 90 dias.
Trata as seguintes colunas:
+ data de cadastro do POC
+ código da ATM onde ocorreu o POC
+ código da agência onde ocorreu o POC

Identifica quais eventos que aconteceram nas agências e na data que saíram da faixa de percentil. estes eventos serão os **Marcadores de POC**.

In [193]:
rdd_poc = sc.textFile(poc_in)
print ("\n\nNúmero de linhas carregadas: " + str("%d" % (rdd_poc.count())))

header = rdd_poc.first()
# pegando data inicial de comprometimento ao inves de data_cad, pois analise exploratoria mostrou que tem
# mais sentido usar esta data
df_poc = rdd_poc \
    .filter(lambda line: line != header) \
    .map(lambda k: k.split(",")) \
    .map(lambda p: (p[0],p[2],p[4], p[8])) \
    .toDF(['data_cad', 
            'atm',
            'agencia',
            'golpe'
        ])

#    .map(lambda p: (p[4],p[2],p[5], p[9])) \
df_poc = df_poc.withColumn('data_cad', funcStrToDate_dmyyyy(col('data_cad'))) 

df_poc = df_poc \
    .withColumn('atm', df_poc["atm"].cast(StringType()))\
    .withColumn('agencia', df_poc["agencia"].cast(StringType())) \
    .withColumn('data_cad', df_poc["data_cad"].cast(TimestampType())) \
    .withColumn('golpe', df_poc["golpe"].cast(StringType())) \

# FILTRANDO POCS SOMENTE DOS ULTIMOS 90 DIAS E SOMENTE PARA GOLPE DE RETENCAO DE CARTAO
df_poc = df_poc.where("CAST(data_cad AS INT) BETWEEN {0} AND {1}"\
                                .format(dt_sql_90,dt_sql_2)).orderBy("data_cad")
#df_eventlog = df_eventlog.where("CAST(data_cad AS INT) >= {0}".format(dt_sql)).orderBy("dh_even")
df_poc.limit(10).toPandas()

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.collectAndServe.
: org.apache.hadoop.mapred.InvalidInputException: Input path does not exist: hdfs://nameservice1/temporario/V6_BASE_POC_ATM_060916.csv
	at org.apache.hadoop.mapred.FileInputFormat.singleThreadedListStatus(FileInputFormat.java:287)
	at org.apache.hadoop.mapred.FileInputFormat.listStatus(FileInputFormat.java:229)
	at org.apache.hadoop.mapred.FileInputFormat.getSplits(FileInputFormat.java:315)
	at org.apache.spark.rdd.HadoopRDD.getPartitions(HadoopRDD.scala:207)
	at org.apache.spark.rdd.RDD$$anonfun$partitions$2.apply(RDD.scala:239)
	at org.apache.spark.rdd.RDD$$anonfun$partitions$2.apply(RDD.scala:237)
	at scala.Option.getOrElse(Option.scala:120)
	at org.apache.spark.rdd.RDD.partitions(RDD.scala:237)
	at org.apache.spark.rdd.MapPartitionsRDD.getPartitions(MapPartitionsRDD.scala:35)
	at org.apache.spark.rdd.RDD$$anonfun$partitions$2.apply(RDD.scala:239)
	at org.apache.spark.rdd.RDD$$ano

In [48]:
#    .map(lambda p: (p[4],p[2],p[5], p[9])) \
df_poc = df_poc.withColumn('data_cad', funcStrToDate_dmyyyy(col('data_cad'))) 

df_poc = df_poc \
    .withColumn('atm', df_poc["atm"].cast(StringType()))\
    .withColumn('agencia', df_poc["agencia"].cast(StringType())) \
    .withColumn('data_cad', df_poc["data_cad"].cast(TimestampType())) \
    .withColumn('golpe', df_poc["golpe"].cast(StringType())) \

# FILTRANDO POCS SOMENTE DOS ULTIMOS 90 DIAS E SOMENTE PARA GOLPE DE RETENCAO DE CARTAO
df_poc = df_poc.where("CAST(data_cad AS INT) BETWEEN {0} AND {1}"\
                                .format(dt_sql_90,dt_sql_2)).orderBy("data_cad")
#df_eventlog = df_eventlog.where("CAST(data_cad AS INT) >= {0}".format(dt_sql)).orderBy("dh_even")
df_poc.limit(10).toPandas()



Número de linhas carregadas: 1019


Py4JJavaError: An error occurred while calling o522.javaToPython.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 1 in stage 70.0 failed 4 times, most recent failure: Lost task 1.3 in stage 70.0 (TID 5022, srvbdrprlbr04.santanderbr.corp): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/produtos/bdr/parcel/parcels/CDH-5.5.1-1.cdh5.5.1.p0.11/lib/spark/python/pyspark/worker.py", line 111, in main
    process()
  File "/produtos/bdr/parcel/parcels/CDH-5.5.1-1.cdh5.5.1.p0.11/lib/spark/python/pyspark/worker.py", line 106, in process
    serializer.dump_stream(func(split_index, iterator), outfile)
  File "/produtos/bdr/parcel/parcels/CDH-5.5.1-1.cdh5.5.1.p0.11/lib/spark/python/pyspark/serializers.py", line 263, in dump_stream
    vs = list(itertools.islice(iterator, batch))
  File "/produtos/bdr/parcel/parcels/CDH-5.5.1-1.cdh5.5.1.p0.11/lib/spark/python/pyspark/sql/functions.py", line 1417, in <lambda>
    func = lambda _, it:

In [106]:
#rdd_poc.collect()

In [107]:
# tirando zeros a esquerda e truncando codigos de agencia e atm
df_poc = df_poc\
    .withColumn('atm', trim(df_poc.atm))\
    .withColumn('agencia', trim(df_poc.agencia))\
    .withColumn('agencia', regexp_replace('agencia', '^0+(?!$)', '')) # removendo zero `esquerda
#df_poc.limit(10).toPandas()

In [108]:
print ("\nNúmero de linhas carregadas: " + str("%d" % (df_poc.count())))
print ("\nNúmero de linhas após retirar NAs: " + str("%d" % (df_poc.dropna().count())))


Número de linhas carregadas: 384

Número de linhas após retirar NAs: 384


## Preparando dados processados
Marcando como POC as entradas do log para os eventos marcadores 

In [109]:
# APROXIMACAO ABAIXO PARA EVITAR PROBLEMAS NO withColumns (TIRAR SE NAO USAR PIVOT DE PANDAS)
# criar coluna de indice de proximidade da mediana (iproxmed)
# calculado como número de ocorrências real diaria/mediana diaria de eventos por agencia e atm
# If you use deimcal, you need to import

#from decimal import getcontext, Decimal
# Set the precision.
#getcontext().prec = 3

#format(math.pi, '.2f')
# mudar o arredondamento abaixo para ver se afeta o modelo
df_feat = df_eventlog2\
    .withColumn('iproxmed', (col("nr_eventos")/col("mediana"))*1000)
df_feat.limit(10).toPandas()

nr_idef_secu cd_eqpt nr_sequ_even    dh_even  nr_eventos  mediana  \
0         1217     156         1642 2016-05-31           1      1.0   
1         1227     155         2982 2016-07-07           1      2.0   
2         1227     155         2982 2016-07-14           2      2.0   
3         1227     155         2982 2016-06-03           1      2.0   
4         1227     155         2982 2016-08-08           2      2.0   
5         1227     155         2982 2016-08-01           1      2.0   
6         1227     155         2982 2016-07-28           1      2.0   
7         1227     155         2982 2016-07-04           1      2.0   
8         1227     155         2982 2016-05-16           1      2.0   
9         1227     155         2982 2016-06-24           1      2.0   

   percentil_max  iproxmed  
0            1.0    1000.0  
1            2.0     500.0  
2            2.0    1000.0  
3            2.0     500.0  
4            2.0    1000.0  
5            2.0     500.0  
6            2.0     500.0  
7            2.0     500.0  
8            2.0     500.0  
9            2.0     500.0

In [110]:
# transpondo tipo de evento para gerar features
df_feat = df_feat\
    .drop(df_feat["mediana"])

In [111]:
# voltar a usar aqui o pandas para transposicao pois erro no sqlcontext
pd_pivot = df_feat\
    .select('dh_even', 'nr_idef_secu', 'cd_eqpt', 'nr_sequ_even', 'iproxmed').toPandas()

In [112]:
pd_feat = pd.pivot_table(pd_pivot,index=["nr_idef_secu","cd_eqpt","dh_even"],
                         columns=["nr_sequ_even"], margins=False, values=["iproxmed"], fill_value=0).reset_index()

pd_feat.columns = [c[1] if c[1] else c[0] for c in pd_feat.columns.tolist()]
pd_feat = pd_feat.assign(data_even = lambda x: x['dh_even'].dt.strftime('%Y-%m-%d') ).drop('dh_even', axis=1)
pd_feat

nr_idef_secu cd_eqpt  1641  1642         2034    2982    3260  3425  \
0              105   10001   0.0   0.0     0.000000     0.0  1000.0   0.0   
1              105   10001   0.0   0.0     0.000000     0.0  1000.0   0.0   
2              105   10001   0.0   0.0     0.000000     0.0  1000.0   0.0   
3              105   10001   0.0   0.0     0.000000     0.0  1000.0   0.0   
4              105   10001   0.0   0.0     0.000000     0.0  1000.0   0.0   
5              105   10001   0.0   0.0     0.000000     0.0  1000.0   0.0   
6              105   10001   0.0   0.0     0.000000     0.0  1000.0   0.0   
7              105   10001   0.0   0.0     0.000000     0.0  1000.0   0.0   
8              105   10001   0.0   0.0     0.000000     0.0  1000.0   0.0   
9              105   10001   0.0   0.0     0.000000     0.0  1000.0   0.0   
10             105   10001   0.0   0.0     0.000000     0.0  1000.0   0.0   
11             105   10001   0.0   0.0     0.000000     0.0  1000.0   0.0   
12             105   10001   0.0   0.0     0.000000     0.0  1000.0   0.0   
13             105   10001   0.0   0.0     0.000000     0.0  1000.0   0.0   
14             105   10001   0.0   0.0     0.000000     0.0  2000.0   0.0   
15             105   10001   0.0   0.0     0.000000     0.0  1000.0   0.0   
16             105   10001   0.0   0.0     0.000000     0.0  1000.0   0.0   
17             105   10001   0.0   0.0     0.000000     0.0  1000.0   0.0   
18             105   10001   0.0   0.0     0.000000     0.0  1000.0   0.0   
19             105   10001   0.0   0.0     0.000000     0.0  1000.0   0.0   
20             105   10001   0.0   0.0     0.000000     0.0  1000.0   0.0   
21             105   10001   0.0   0.0     0.000000     0.0  1000.0   0.0   
22             105   10001   0.0   0.0     0.000000     0.0  1000.0   0.0   
23             105   10001   0.0   0.0     0.000000     0.0  2000.0   0.0   
24             105   10001   0.0   0.0     0.000000     0.0  1000.0   0.0   
25             105   10001   0.0   0.0     0.000000     0.0  1000.0   0.0   
26             105   10001   0.0   0.0     0.000000     0.0  1000.0   0.0   
27             105   10001   0.0   0.0     0.000000     0.0  1000.0   0.0   
28             105   10001   0.0   0.0     0.000000     0.0  1000.0   0.0   
29             105   10001   0.0   0.0     0.000000     0.0  1000.0   0.0   
...            ...     ...   ...   ...          ...     ...     ...   ...   
73382          990     153   0.0   0.0   428.571429     0.0     0.0   0.0   
73383          990     153   0.0   0.0   428.571429     0.0     0.0   0.0   
73384          990     153   0.0   0.0     0.000000  1000.0     0.0   0.0   
73385          990     153   0.0   0.0     0.000000     0.0     0.0   0.0   
73386          990     153   0.0   0.0   142.857143     0.0     0.0   0.0   
73387          990     153   0.0   0.0   142.857143  1000.0     0.0   0.0   
73388          990     153   0.0   0.0     0.000000     0.0     0.0   0.0   
73389          990     153   0.0   0.0   142.857143     0.0     0.0   0.0   
73390          990     153   0.0   0.0   142.857143  1000.0     0.0   0.0   
73391          990     153   0.0   0.0     0.000000     0.0     0.0   0.0   
73392          990     153   0.0   0.0   428.571429     0.0     0.0   0.0   
73393          990     153   0.0   0.0   571.428571     0.0     0.0   0.0   
73394          990     153   0.0   0.0   285.714286     0.0     0.0   0.0   
73395          990     153   0.0   0.0   285.714286     0.0     0.0   0.0   
73396          990     153   0.0   0.0   285.714286  1000.0     0.0   0.0   
73397          990     153   0.0   0.0     0.000000     0.0     0.0   0.0   
73398          990     153   0.0   0.0   285.714286     0.0     0.0   0.0   
73399          990     153   0.0   0.0   428.571429     0.0     0.0   0.0   
73400          990     153   0.0   0.0   428.571429     0.0     0.0   0.0   
73401          990     153   0.0   0.0   428.571429     0.0     0.0   0.0   
73

In [113]:
pd_feat.columns

Index([u'nr_idef_secu',      u'cd_eqpt',         u'1641',         u'1642',
               u'2034',         u'2982',         u'3260',         u'3425',
               u'4266',         u'4268',         u'4269',          u'899',
          u'data_even'],
      dtype='object')

In [114]:
df_feat_alt = sqlCtx.createDataFrame(pd_feat)

In [115]:
df_feat_alt.limit(10).toPandas()

nr_idef_secu cd_eqpt  1641  1642  2034  2982    3260  3425  4266  4268  \
0         3246   10001   0.0   0.0   0.0   0.0   500.0   0.0   0.0   0.0   
1         3246   10001   0.0   0.0   0.0   0.0  1000.0   0.0   0.0   0.0   
2         3246   10001   0.0   0.0   0.0   0.0   500.0   0.0   0.0   0.0   
3         3246   10001   0.0   0.0   0.0   0.0  1000.0   0.0   0.0   0.0   
4         3246   10001   0.0   0.0   0.0   0.0   500.0   0.0   0.0   0.0   
5         3246   10001   0.0   0.0   0.0   0.0   500.0   0.0   0.0   0.0   
6         3246   10001   0.0   0.0   0.0   0.0   500.0   0.0   0.0   0.0   
7         3246   10001   0.0   0.0   0.0   0.0   500.0   0.0   0.0   0.0   
8         3246   10001   0.0   0.0   0.0   0.0   500.0   0.0   0.0   0.0   
9         3246   10001   0.0   0.0   0.0   0.0   500.0   0.0   0.0   0.0   

   4269  899   data_even  
0   0.0  0.0  2016-06-23  
1   0.0  0.0  2016-06-24  
2   0.0  0.0  2016-06-28  
3   0.0  0.0  2016-06-29  
4   0.0  0.0  2016-06-30  
5   0.0  0.0  2016-07-01  
6   0.0  0.0  2016-07-02  
7   0.0  0.0  2016-07-05  
8   0.0  0.0  2016-07-06  
9   0.0  0.0  2016-07-07

In [116]:
df_feat_alt.printSchema()

root
 |-- nr_idef_secu: string (nullable = true)
 |-- cd_eqpt: string (nullable = true)
 |-- 1641: double (nullable = true)
 |-- 1642: double (nullable = true)
 |-- 2034: double (nullable = true)
 |-- 2982: double (nullable = true)
 |-- 3260: double (nullable = true)
 |-- 3425: double (nullable = true)
 |-- 4266: double (nullable = true)
 |-- 4268: double (nullable = true)
 |-- 4269: double (nullable = true)
 |-- 899: double (nullable = true)
 |-- data_even: string (nullable = true)



In [117]:
# testar abaixo se este mesmo evento se encontra no seletor de poc e porque nao aparece aqui
#df_feat.filter(col('nr_sequ_even') == "1746").count()

In [118]:
# na verdade nao precisa executar o codigo abaixo na query de transposicao
# criando lista somente com pokeventos que nao existem no log
# temporariamente carregando l_evenpoc somente com poucos eventos por falta de memoria
# ver em producao se roda
#l_evenpoc = ["5401","1746","666","1903","2034","2930","4236","6732", "1588"]
#existe = [even if df_feat.filter(col('nr_sequ_even') == even).count() == 0 else None for even in l_evenpoc]
#existe

In [119]:
# Provavel Bug no Pyspark impede abordagem abaixo para nao usar  pivot do Pandas (ao corrigir, substituir)

#df_poc.registerTempTable("poc")

#df_feat = sqlContext.sql("select nr_idef_secu,cd_eqpt,dh_even,\
#                        max(ev4266) as ev4266,\
#                        max(ev5490) as ev5490,\
#                        max(ev899) as ev899,\
#                        max(ev2034) as ev2034,\
#                        max(ev4267) as ev4267,\
#                        max(ev4268) as ev4268,\
#                        max(ev2982) as ev2982,\
#                        max(ev3260) as ev3260\
#                    from (select nr_idef_secu,cd_eqpt,dh_even,\
#                                  case when nr_sequ_even='4266' then iproxmed else 0 end ev4266,\
#                                  case when nr_sequ_even='5490' then iproxmed else 0 end ev5490,\
#                                  case when nr_sequ_even='899' then iproxmed else 0 end ev899,\
#                                  case when nr_sequ_even='2034' then iproxmed else 0 end ev2034,\
#                                  case when nr_sequ_even='4267' then iproxmed else 0 end ev4267,\
#                                  case when nr_sequ_even='4268' then iproxmed else 0 end ev4268,\
#                                  case when nr_sequ_even='2982' then iproxmed else 0 end ev2982,\
#                                  case when nr_sequ_even='3260' then iproxmed else 0 end ev3260 \
#                             from df_feat) x \
#                     group by nr_idef_secu,cd_eqpt,dh_even")

#df_feat_alt.limit(10).toPandas()

#l_features = ['ev4266', 'ev5490', 'ev899', 'ev2034', 'ev4267', 'ev4268', 'ev2982', 'ev3260']

In [120]:
#pd_feat.columns = [c[1] if c[1] else c[0] for c in pd_feat.columns.tolist()]

In [121]:
#pd_feat

In [122]:
#c = pd_feat.columns.tolist
#c[1]

In [123]:
# criar coluna com data como string para posterior transformação para dataframe
#pd_feat = pd_feat.assign(data_even = lambda x: x['dh_even'].dt.strftime('%Y-%m-%d') ).drop('dh_even', axis=1)
#pd_feat

In [124]:
# salva como csv pois conversao para Dataframe Spark apresenta erro. provavel falta de memoria
#pd_feat.to_csv(tabelao_out)

In [125]:
#pd_feat.columns

In [126]:
#df_feat_alt = sqlCtx.createDataFrame(pd_feat)
# FALTA converter data em timedate de novo (converter para string no Pandas antes)

In [127]:
#df_feat_alt.take(5)

In [128]:
#df_feat_alt.limit(10).toPandas()

In [129]:
# OBS: se já tiver o tabelao_alt gerado, pode rodar a partir daqui!!!!!
#############################################################################
# AQUI TIVE QUE FAZER ESTA GAMBIARRA: salvar como csv e alguma manipulaçao (acertar cabeçalho) para depois ler
# até resolver o problema de memoria gerado por fazer o pivot no Pandas
#rdd_feat_alt = sc.textFile(tabelao_in)

In [130]:
#rdd_feat_alt.first()
#df = sqlContext.load(source="com.databricks.spark.csv", header="true", path = "cars.csv")

In [131]:
#print ("\n\nNúmero de linhas carregadas: " + str("%d" % (rdd_feat_alt.count())))

In [132]:
#header = rdd_feat_alt.first()
#schemaString = header
#fields = [StructField(field_name, StringType(), False) for field_name in schemaString.split(',')]
#schema = StructType(fields)

#rdd_feat_alt = rdd_feat_alt \
##    .filter(lambda line: line != header) \
#    .map(lambda k: k.split(","))

#df_feat_alt = sqlContext.createDataFrame(rdd_feat_alt, schema)

In [133]:
#df_feat_alt.limit(10).toPandas()

In [134]:
#df_feat_alt.printSchema()

In [135]:
df_feat_alt = df_feat_alt.withColumn('data_even', df_feat_alt["data_even"].cast(TimestampType()))
#df_feat_alt.limit(10).toPandas()

In [136]:
df_feat_alt.printSchema()

root
 |-- nr_idef_secu: string (nullable = true)
 |-- cd_eqpt: string (nullable = true)
 |-- 1641: double (nullable = true)
 |-- 1642: double (nullable = true)
 |-- 2034: double (nullable = true)
 |-- 2982: double (nullable = true)
 |-- 3260: double (nullable = true)
 |-- 3425: double (nullable = true)
 |-- 4266: double (nullable = true)
 |-- 4268: double (nullable = true)
 |-- 4269: double (nullable = true)
 |-- 899: double (nullable = true)
 |-- data_even: timestamp (nullable = true)



In [137]:
df_feat_alt = df_feat_alt\
    .withColumn('nr_idef_secu', trim(df_feat_alt.nr_idef_secu))
#df_feat_alt.printSchema()

In [138]:
#df_feat_alt.printSchema()

In [139]:
# primeiro left_join df_feat_alt + df_poc
    # depois para agencia == cd_secu and cada coluna evento > 1 = POC
df_tidy = df_feat_alt \
    .join(df_poc, (df_feat_alt['nr_idef_secu'] == df_poc['agencia']) &
            (df_feat_alt['data_even'] == df_poc['data_cad']), 'left')

In [140]:
# marca label de ocorrencia de POC
df_tidy = df_tidy.withColumn('label',
            when(col("agencia") == col("nr_idef_secu"), 1.0)\
            .otherwise(0.0))\
            .drop(df_tidy["agencia"]).drop(df_tidy["atm"]).drop(df_tidy["data_cad"])
                            
#df_tidy.limit(10).toPandas()

In [141]:
#df_tidy.orderBy(col('agencia').desc())\
#    .filter(col('agencia') == col('nr_idef_secu'))\
#    .select('agencia').distinct().toPandas()
# 53 agências distintas com POC

In [142]:
df_tidy.limit(10).toPandas()

nr_idef_secu cd_eqpt  1641  1642         2034  2982    3260  3425  4266  \
0          105   10001   0.0   0.0     0.000000   0.0  1000.0   0.0   0.0   
1          105     151   0.0   0.0     0.000000   0.0     0.0   0.0   0.0   
2          105     152   0.0   0.0     0.000000   0.0     0.0   0.0   0.0   
3          105     153   0.0   0.0   317.460317   0.0     0.0   0.0   0.0   
4          105     154   0.0   0.0     0.000000   0.0     0.0   0.0   0.0   
5          105     156   0.0   0.0   243.902439   0.0     0.0   0.0   0.0   
6          105     158   0.0   0.0   533.333333   0.0     0.0   0.0   0.0   
7          105     159   0.0   0.0   204.081633   0.0     0.0   0.0   0.0   
8          105     160   0.0   0.0  1166.666667   0.0     0.0   0.0   0.0   
9          105     161   0.0   0.0  1200.000000   0.0     0.0   0.0   0.0   

   4268         4269  899  data_even golpe  label  
0   0.0     0.000000  0.0 2016-06-07  None    0.0  
1   0.0   500.000000  0.0 2016-06-07  None    0.0  
2   0.0  1000.000000  0.0 2016-06-07  None    0.0  
3   0.0     0.000000  0.0 2016-06-07  None    0.0  
4   0.0   500.000000  0.0 2016-06-07  None    0.0  
5   0.0  1000.000000  0.0 2016-06-07  None    0.0  
6   0.0   500.000000  0.0 2016-06-07  None    0.0  
7   0.0   500.000000  0.0 2016-06-07  None    0.0  
8   0.0  1000.000000  0.0 2016-06-07  None    0.0  
9   0.0   333.333333  0.0 2016-06-07  None    0.0

In [145]:
#l_aux = l_evenpoc[0:10]
#for even in l_aux:
#df_tidy_dbl = df_tidy\
#       .withColumn('2422', df_tidy['2422'].cast(DoubleType()))\
#        .withColumn('1489', df_tidy['1489'].cast(DoubleType()))\
#        .withColumn('679', df_tidy['679'].cast(DoubleType()))\
#        .withColumn('723', df_tidy['723'].cast(DoubleType()))\
#        .withColumn('724', df_tidy['724'].cast(DoubleType()))\
#        .withColumn('2761', df_tidy['2761'].cast(DoubleType()))\
#        .withColumn('3814', df_tidy['3814'].cast(DoubleType()))\
#        .withColumn('3814', df_tidy['3814'].cast(DoubleType()))\
#        .withColumn('2267', df_tidy['2267'].cast(DoubleType()))\
#        .withColumn('6778', df_tidy['6778'].cast(DoubleType()))
df_tidy_dbl = df_tidy

In [146]:
df_tidy_dbl.printSchema()

root
 |-- nr_idef_secu: string (nullable = true)
 |-- cd_eqpt: string (nullable = true)
 |-- 1641: double (nullable = true)
 |-- 1642: double (nullable = true)
 |-- 2034: double (nullable = true)
 |-- 2982: double (nullable = true)
 |-- 3260: double (nullable = true)
 |-- 3425: double (nullable = true)
 |-- 4266: double (nullable = true)
 |-- 4268: double (nullable = true)
 |-- 4269: double (nullable = true)
 |-- 899: double (nullable = true)
 |-- data_even: timestamp (nullable = true)
 |-- golpe: string (nullable = true)
 |-- label: double (nullable = false)



In [147]:
df_tidy_dbl.groupBy("label").count().orderBy("label").show()

+-----+-----+
|label|count|
+-----+-----+
|  0.0|72484|
|  1.0|  997|
+-----+-----+



In [148]:
# tabelao com labels
#df_tidy.toPandas().to_csv(tabelao_label)

In [149]:
#df_tidy.printSchema()

In [150]:
# ATANCAO: ver se falta fazer antes
#df_tidy = df_feat_alt\
#   .withColumn('data_even', df_feat_alt["data_even"].cast(TimestampType()))\
#    .withColumn('label', df_feat_alt["label"].cast(DoubleType()))

In [151]:
#df_tidy.limit(10).toPandas()

In [152]:
#df_tidy.printSchema()

In [153]:
# salvando tabelao com label
#df_tidy.toPandas().to_csv(tabelao_final)

## Modelagem
O modelo usado será o de Random Forest, implementando pipelines da library ML do Spark.
A partir da amostra de dados será selecionado um subset com 75% dos dados para treinar o modelo. 

### Treino do Modelo
Prepara a amostra de acordo com o formato de entrada de label e vetor de features esperados pelo modelo.

In [1]:
# MLLIB 
from pyspark.mllib.tree import RandomForest
from pyspark.mllib.util import MLUtils

# Load and parse the data file into an RDD of LabeledPoint.
#data = MLUtils.loadLibSVMFile(sc, 'data/mllib/sample_libsvm_data.txt')
# Split the data into training and test sets (30% held out for testing)
(trainingData, testData) = df_tidy_rf.randomSplit([0.7, 0.3])

# Train a RandomForest model.
#  Empty categoricalFeaturesInfo indicates all features are continuous.
#  Note: Use larger numTrees in practice.
#  Setting featureSubsetStrategy="auto" lets the algorithm choose.
model = RandomForest.trainClassifier(trainingData, numClasses=2, categoricalFeaturesInfo={},
                                     numTrees=3, featureSubsetStrategy="auto",
                                     impurity='gini', maxDepth=4, maxBins=32)

# Evaluate model on test instances and compute test error
predictions = model.predict(testData.map(lambda x: x.features))
labelsAndPredictions = testData.map(lambda lp: lp.label).zip(predictions)
testErr = labelsAndPredictions.filter(lambda (v, p): v != p).count() / float(testData.count())
print('Test Error = ' + str(testErr))
print('Learned classification forest model:')
print(model.toDebugString())

NameError: name 'df_tidy_rf' is not defined

In [154]:
from pyspark.mllib.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

from pyspark.ml import Pipeline
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [155]:
df_tidy_dbl.printSchema()

root
 |-- nr_idef_secu: string (nullable = true)
 |-- cd_eqpt: string (nullable = true)
 |-- 1641: double (nullable = true)
 |-- 1642: double (nullable = true)
 |-- 2034: double (nullable = true)
 |-- 2982: double (nullable = true)
 |-- 3260: double (nullable = true)
 |-- 3425: double (nullable = true)
 |-- 4266: double (nullable = true)
 |-- 4268: double (nullable = true)
 |-- 4269: double (nullable = true)
 |-- 899: double (nullable = true)
 |-- data_even: timestamp (nullable = true)
 |-- golpe: string (nullable = true)
 |-- label: double (nullable = false)



In [158]:
# rodando para as features com mais relevância já analisadas
#assemblerTrain = VectorAssembler(
#    inputCols=["1641", "1642", "2034", "2982", "3260", "3425", "4266", "4268", "4269", "899"],
#   outputCol="features")
assemblerTrain = VectorAssembler(
#    inputCols=l_features,
    inputCols=["1641", "1642", "2034", "2982", "3260", "3425", "4266", "4268", "4269", "899"],
   outputCol="features")

#               ,"1617", "1689","2433", "1746", "1568","883", "2163"],
 

In [159]:
df_tmp = assemblerTrain.transform(df_tidy_dbl)

In [160]:
df_tidy_rf = df_tmp.select("nr_idef_secu", "cd_eqpt", "label", "features")
df_tidy_rf.printSchema()

root
 |-- nr_idef_secu: string (nullable = true)
 |-- cd_eqpt: string (nullable = true)
 |-- label: double (nullable = false)
 |-- features: vector (nullable = true)



In [161]:
df_tidy_rf.take(5)

[Row(nr_idef_secu=u'1053', cd_eqpt=u'10001', label=0.0, features=SparseVector(10, {4: 1000.0})),
 Row(nr_idef_secu=u'1053', cd_eqpt=u'151', label=0.0, features=SparseVector(10, {2: 180.1802})),
 Row(nr_idef_secu=u'1053', cd_eqpt=u'152', label=0.0, features=SparseVector(10, {2: 71.4286})),
 Row(nr_idef_secu=u'1053', cd_eqpt=u'153', label=0.0, features=SparseVector(10, {2: 1000.0, 3: 500.0})),
 Row(nr_idef_secu=u'1053', cd_eqpt=u'154', label=0.0, features=SparseVector(10, {3: 500.0}))]

In [162]:
# Index labels, adding metadata to the label column.
# Fit on whole dataset to include all labels in index.
labelIndexer = StringIndexer(inputCol="label", outputCol="indexedLabel").fit(df_tidy_rf)
#labelIndexer = StringIndexer(inputCol="label", outputCol="indexedLabel").fit(trainingData_rf)

In [163]:
# Automatically identify categorical features, and index them.
# We specify maxCategories so features with > 2 distinct values are treated as continuous.
featureIndexer =\
    VectorIndexer(inputCol="features", outputCol="indexedFeatures", maxCategories=2).fit(df_tidy_rf)
#featureIndexer =\
#    VectorIndexer(inputCol="features", outputCol="indexedFeatures", maxCategories=2).fit(trainingData_rf)

In [164]:
# Split the data into training and test sets (30% held out for testing)
(trainingData_rf, testData_rf) = df_tidy_rf.randomSplit([0.7, 0.3])

In [165]:
trainingData_rf.groupBy("label").count().orderBy("label").show()

+-----+-----+
|label|count|
+-----+-----+
|  0.0|50729|
|  1.0|  716|
+-----+-----+



In [166]:
testData_rf.groupBy("label").count().orderBy("label").show()

+-----+-----+
|label|count|
+-----+-----+
|  0.0|21755|
|  1.0|  281|
+-----+-----+



Separa a amostra de forma aleatória em dados de treino (70% da amostra) e de teste (30% da amostra, para posterior avaliação do modelo).

In [167]:
# Train a Random Forest model.
rf = RandomForestClassifier(labelCol="indexedLabel", featuresCol="indexedFeatures")
#dt = DecisionTreeClassifier(labelCol="indexedLabel", featuresCol="indexedFeatures")

In [168]:
# Chain indexers and tree in a Pipeline
pipeline = Pipeline(stages=[labelIndexer, featureIndexer, rf])

In [169]:
model = pipeline.fit(trainingData_rf)

In [475]:
#pipeline = Pipeline(stages=[rf])
# Search through decision tree's maxDepth parameter for best model
#from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
#from pyspark.ml.evaluation import BinaryClassificationEvaluator
#paramGrid = ParamGridBuilder().addGrid(rf.maxDepth, [2,3,4,5,6,7]).build()

#numFolds = 5

In [476]:
#crossval = CrossValidator(
#    estimator=pipeline,
#    estimatorParamMaps=paramGrid,
#    evaluator=evaluator,
#    numFolds=numFolds)

In [477]:
# Fetch best model
#tree_model = model.bestModel.stages[2]
#print tree_model

In [478]:
# Train model.  This also runs the indexers.
#model = pipeline.fit(trainingData_rf)

## Persistência do Modelo
Salva o modelo para posterior uso. O código comentado abaixo deve ser descomentado nas manutenções para testar se modelo foi corretamente salvo.

In [200]:
# SAVE MODEL FOR FUTURE USE (TEST IF SAVED OK)
#model.save(sc, savedModel) # Erro. Funciona com mllib. Pesquisar como salvar pipeline
# persist model to HDFS
#model.save(sc, tabelao_out)
#sc.parallelize(Seq(model), 1).saveAsObjectFile("hdfs:///user/root/linReg.model")

# READ MODEL
#linRegModel = sc.objectFile[LinearRegressionModel]("linReg.model").first()

### Avaliação do Modelo
Avalia o modelo usando os dados separados para teste, obtendo o erro de teste (1 - acurácia), a área sob a curva ROC (AUC) e a taxa Precision/Recall.

In [201]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator
predictions = model.transform(testData_rf)

In [202]:
evaluator = BinaryClassificationEvaluator()

In [203]:
evaluator.explainParams()

'labelCol: label column name (default: label)\nmetricName: metric name in evaluation (areaUnderROC|areaUnderPR) (default: areaUnderROC)\nrawPredictionCol: raw prediction (a.k.a. confidence) column name (default: rawPrediction)'

In [204]:
auroc = evaluator.evaluate(predictions, {evaluator.metricName: "areaUnderROC"})

In [205]:
print("AUC = %g " % (auroc))

AUC = 0.524425 


In [175]:
precrec = evaluator.evaluate(predictions, {evaluator.metricName: "areaUnderPR"})

In [176]:
print("Precision/Recall = %g " % (precrec))

Precision/Recall = 0.0149375 


In [177]:
accuracy = evaluator.evaluate(predictions)
print("Test Error = %g " % (1.0 - accuracy))

Test Error = 0.475212 


In [178]:
#predictions.select

Abaixo são mostradas as primeiras linhas da tabela de previsão gerada pelo modelo

In [207]:
predictions.select("nr_idef_secu", "cd_eqpt", "prediction", "indexedLabel", "features", "probability").take(5)

[Row(nr_idef_secu=u'1053', cd_eqpt=u'151', prediction=0.0, indexedLabel=0.0, features=SparseVector(10, {2: 360.3604, 8: 666.6667}), probability=DenseVector([0.9857, 0.0143])),
 Row(nr_idef_secu=u'1053', cd_eqpt=u'152', prediction=0.0, indexedLabel=0.0, features=SparseVector(10, {2: 428.5714, 3: 333.3333, 8: 1000.0}), probability=DenseVector([0.9859, 0.0141])),
 Row(nr_idef_secu=u'112', cd_eqpt=u'152', prediction=0.0, indexedLabel=1.0, features=SparseVector(10, {2: 600.0, 3: 465.1163, 8: 500.0, 9: 1000.0}), probability=DenseVector([0.9842, 0.0158])),
 Row(nr_idef_secu=u'112', cd_eqpt=u'153', prediction=0.0, indexedLabel=1.0, features=SparseVector(10, {2: 250.0, 3: 370.3704}), probability=DenseVector([0.9885, 0.0115])),
 Row(nr_idef_secu=u'112', cd_eqpt=u'154', prediction=0.0, indexedLabel=1.0, features=SparseVector(10, {2: 666.6667, 3: 487.8049, 8: 454.5455}), probability=DenseVector([0.9887, 0.0113]))]

In [276]:
#x = predictions.map(lambda a: (a[0], a[1], a[7][1]))

In [274]:
from pyspark.mllib.linalg import Vectors
def extract(row):
    return (row.nr_idef_secu, row.cd_eqpt, ) + tuple(float(x) for x in row.probability.values)

x = predictions.select("nr_idef_secu", "cd_eqpt", "probability")
x.rdd.map(extract).toDF(["nr_idef_secu", "cd_eqpt", "probFALSE", "probTRUE" ])


In [294]:
x.first()

KeyboardInterrupt: 

In [291]:
x.toDF(['agencia',
           'atm', 
           'renking'])

TypeError: not supported type: <type 'numpy.float64'>

In [278]:
df_x = sqlCtx.createDataFrame(x)

TypeError: not supported type: <type 'numpy.float64'>

In [ ]:
df_x

In [180]:
# 6. dividir dados em treino e teste
# selecionando sample estratificada
#df_tidy_dbl.groupBy("label").count().orderBy("label").show()
# N: 0.9995   S: 0.000499
# Balanceamento com marcação anterior de report de POC
# N: 0.0110104 S: 0.98898
# Balanceamento com marcação por evento fora do percentil de POC
# N:  S: 

In [181]:
#df_tidy_dbl.groupBy(["nr_idef_secu", "label"]).count().orderBy("nr_idef_secu").show()

In [182]:
# salving the resulting dataframe
#type(poc_predicted)
#poc_predicted.to_csv(previsao)
#df.write.csv('mycsv.csv')

In [183]:
# estratégia
# 1. obter a média diária da agência + atm (trocar para mediana) para cada evento
# 2. do log, somar o número de ocorrências por agencia + atm + evento agrupado por dia
# 3. montar tabelao inicial e criar coluna count/media para ver índice de desvio da média (FEATURE) por agencia/atm
# 4. criar coluna LABEL concatenando com arquivo POC por agencia + data cad (atm nao confiavel no arquivo)
# obs: desvio > 1 indica percentual acima da média e vice-versa
# 5. fazer o pivot como acima

# trocar media por mediana (FALTA)
# usar mediana para gerar o tabelao (ou percentil 75% ou soma contante) (FALTA)
# dividir pelo count para ter a feature percentil/count (FALTA)
# transposicao (filtrar tabelao por agencia + atm e join dois a dois (em loop?))
# filtrar os 3 dias após a data de poc para tirar efeitos de setup   
# 

In [184]:
# matriz de confusão
#poc_predicted

## confusion matrix

In [185]:
def confusion_matrix(prevCol, labelCol):
    if prevCol == 1.0 and labelCol == 1.0:
        return 'TP'
    else:
        if prevCol == 1.0 and labelCol == 0.0:
            return 'FP'
        else:
            if prevCol == 0.0 and labelCol == 0.0:
                return 'TN'
            else:
                if prevCol == 0.0 and labelCol == 1.0:
                    return 'FN'
            
createNewColFromTwo = udf(confusion_matrix, StringType())
df_conf_matr = predictions.withColumn('conf_matr', createNewColFromTwo(predictions['prediction'],
            predictions['label']))

In [186]:
# matriz de confuao
df_rates = df_conf_matr.groupBy("conf_matr").count()
df_rates.toPandas()

conf_matr  count
0        TN  21755
1        FN    281

In [143]:
# Sensibility - habilidade do modelo classificar corretamente um POC
# Probability of being test positive when POC present.
# TP / (TP + FN)
sensitivity = float(df_rates.collect()[0][1]) / (df_rates.collect()[0][1] + df_rates.collect()[2][1])
# Specificity - habilidade do modelo classificar corretamente a ausência de POC 
#Probability of being test negative when POC absent.
# TN / (TN + FP)
specificity = float(df_rates.collect()[1][1]) / (df_rates.collect()[1][1] + df_rates.collect()[3][1])
# precisao
# TP / (TP + FP)
precisao = float(df_rates.collect()[0][1]) / (df_rates.collect()[0][1] + df_rates.collect()[3][1])

print ("\nSentitividade ou Recall: " + str("%.2f" % (sensitivity)))
print ("\nEspecificidade         : " + str("%.2f" % (specificity)))
print ("\nPrecisão               : " + str("%.2f" % (precisao)))

IndexError: list index out of range

In [144]:
#df_rates.toPandas()

In [187]:
df_rates.collect()

[Row(conf_matr=u'TN', count=21755), Row(conf_matr=u'FN', count=281)]

In [188]:
# TESTES
predictions.filter((col("prediction") == 0.0) & (col("label") == 1.0)).count()

324

In [189]:
predictions.filter((col("prediction") == 1.0) & (col("label") == 1.0)).count()

0

In [190]:
predictions.filter((col("prediction") == 1.0) & (col("label") == 0.0)).count()

0

In [191]:
predictions.filter((col("prediction") == 0.0) & (col("label") == 0.0)).count()

21735

In [138]:
# checagem do marcador de POC contra POCs reportados
# somente POC retencao de cartão, D-90
# trazer número de POCs para a chave agencia
# encotradas 52 agências com POC
#df_p = df_poc.groupBy('agencia').count().orderBy('agencia')
#df_p.toPandas()
# agora comparar com o arquivo gerado com label

In [137]:
# foram encontradas agencias de POC nos eventos abaixo filtrados
#df_l = df_feat.where(col('label') == 1.0).groupBy('nr_idef_secu').count().orderBy('nr_idef_secu')
#df_p = df_poc.groupBy('agencia').count().orderBy('agencia')
#df_l.toPandas()

In [136]:
# checar se selecionou realmente somente agencias com POC do log geral
#df_evenpoc.groupBy('nr_idef_secu').count().orderBy('nr_idef_secu').toPandas()

In [134]:
# checar quantas agencias reportadas como POC cairam no log com nova marcação (deveriam aparecer todos )
#df_j = df_l \
#    .join(df_p, (df_l['nr_idef_secu'] == df_p['agencia']), 'inner')
    
    

In [135]:
#df_j.limit(10).toPandas()